## Experiment1
4個の記憶を順番に入れ続け、次の３つの変化を記録する。  
    ・記憶ベクトルの分布  
    ・記憶辞書のマッピング結果  
    ・一時記憶の変化  

### Module definition and importing.

In [1]:
# import modules
import torch
import numpy as np
from logger import *
from MemoryDictionary import MemoryDictionary
from WorkingMemory import WorkingMemory
from torch.utils.tensorboard import SummaryWriter
from log_tool import log_mem_dict

In [2]:
# Memory Generator
class MemGenerator:
    """
    0 -> 1 -> ... -> max_id - 1-> 0 -> ...
    上のように、順番に記憶IDを出力します
    """

    def __init__(self, max_id:int) -> None:
        assert max_id > 0
        self.max_id = max_id
        self.current_id = 0

    def get_mem(self) -> int:
        out = self.current_id
        self.current_id = (self.current_id + 1) % self.max_id
        return out
mem_g = MemGenerator(4)

### settings

In [3]:
MAX_ID = 4
ALPHA = 2
NUM_DIM = 6
NUM_STEPS = 32
LOG_INTERVAL = 1
DEVICE = torch.device("cuda")
DTYPE = torch.float
CONNECTION_MARGIN = 0.1
MAX_LR = 10.0
NUM_SEARCH = 10

### Prepair

In [7]:
setLogger(__name__,INFO)
mem_dict= MemoryDictionary(MAX_ID,NUM_DIM,DEVICE,DTYPE,CONNECTION_MARGIN,MAX_LR,NUM_SEARCH)
work_mem = WorkingMemory(ALPHA)
mem_gentor = MemGenerator(MAX_ID)
writer = SummaryWriter(log_dir="runs/experiment1")

### Run

In [8]:
shutdown = False
info("Start simulation!")
steps = 0
while not shutdown:
    mem = mem_gentor.get_mem()
    work_mem.add(mem)
    pairs = work_mem.create_pairs()
    traced = mem_dict.trace(work_mem.memories)
    work_mem.add(traced)
    for p in pairs:
        mem_dict.connect(p[0],p[1])
    
    if steps % LOG_INTERVAL == 0:
        log_mem_dict(writer,mem_dict,p)

    steps += 1
    if steps >= NUM_STEPS:
        shutdown = True
        info("End simulation")


[2022-01-21 17:16:23,830][INFO][__main__] | Start simulation!
[2022-01-21 17:16:23,830][INFO][__main__] | Start simulation!
[2022-01-21 17:16:23,839][INFO][__main__] | Current working memories are [0]
[2022-01-21 17:16:23,839][INFO][__main__] | Current working memories are [0]
[2022-01-21 17:16:23,862][INFO][__main__] | Current working memories are [0 1]
[2022-01-21 17:16:23,862][INFO][__main__] | Current working memories are [0 1]
[2022-01-21 17:16:23,878][INFO][__main__] | Current working memories are [0 2]
[2022-01-21 17:16:23,878][INFO][__main__] | Current working memories are [0 2]
[2022-01-21 17:16:23,893][INFO][__main__] | Current working memories are [0 3]
[2022-01-21 17:16:23,893][INFO][__main__] | Current working memories are [0 3]
[2022-01-21 17:16:23,915][INFO][__main__] | Current working memories are [0 3]
[2022-01-21 17:16:23,915][INFO][__main__] | Current working memories are [0 3]
[2022-01-21 17:16:23,927][INFO][__main__] | Current working memories are [0 3]
[2022-01-21